In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/change_point/

/content/drive/MyDrive/change_point


In [ ]:
import numpy as np
from numpy.random import randn
from numpy.random import laplace
from sklearn.linear_model import LogisticRegression
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cvx
from sklearn.metrics.pairwise import pairwise_distances


sigma_1  = 0.000176
sigma_2 = 0.013938
sigma_3 = 0.013542
sigma_4 = 0.057944

%matplotlib inline

### Online change point detection procedure (polynomials)


In [ ]:
# Auxiliary function
# Computation of design matrix based on (1, x, x**2, ..., x**(p-1))
#
# X -- array of univariate observations
#
# p -- positive integer
#
def compute_design(X, p):
    
    n = X.shape[0]
    Psi = np.power(np.outer(X, np.ones(p)), np.outer(np.ones(n), np.arange(p)))
    # print(Psi.shape)
    
    return Psi

In [ ]:
# Auxiliary function
# Computation of the best fitting parameter theta
# under the hypothesis that tau is the true change point
#
# Psi -- (n x p)-array, design matrix 
#
# tau -- change point candidate
#
def compute_theta(Psi, tau):
    
    # Sample size
    t = Psi.shape[0]
    
    # Create "virtual" labels
    Y = np.append(np.ones(tau), -np.ones(t - tau))
    
    lr = LogisticRegression(penalty='none', fit_intercept=False, tol=1e-2,\
                            solver='newton-cg', class_weight='balanced', n_jobs=-1)
    lr.fit(Psi, Y)
    theta = (lr.coef_).reshape(-1)
    
    return theta

In [ ]:
# Computation of the test statistic
#
# X -- array of univariate observations
#
# p -- positive integer (used for basis construction)
#
def compute_test_stat_linear(X, p, t_min=20, n_out_min=10, B=10):
    
    # Sample size
    n = X.shape[0]
    
    # Compute design matrix
    Psi = compute_design(X, p)
    
    # Initialization
    T = np.zeros((n, n))
    
    for t in range(t_min, n):
        
        D = np.zeros(t)
        
        for tau in range(n_out_min, t-n_out_min):
            
            # Compute the best fitting parameter theta
            theta = compute_theta(Psi[:t, :], tau)
            Z = Psi[:t, :] @ theta
            
            # Use thresholding to avoid numerical issues
            Z = np.minimum(Z, B)
            Z = np.maximum(Z, -B)
            
            D[:tau] = 2 / (1 + np.exp(-Z[:tau]))
            D[tau:] = 2 / (1 + np.exp(Z[tau:]))
            D = np.log(D)
            
            # Compute statistics for each t
            # and each change point candidate tau
            T[tau, t] = tau * (t - tau) / t * (np.mean(D[:tau]) + np.mean(D[tau:]))
            
    # Array of test statistics
    S = np.max(T, axis=0)
    
    return S

### Online change point detection procedure (Fourier basis)


In [ ]:
def compute_design_Fourier(X, p):
    res = np.zeros((p, X.shape[0]))
    res[0] = np.ones(X.shape[0]) / np.sqrt(2)
    T = 1
    for i in range(1, p):
        if (i // 2 == 0):
            res[i] = np.sin(X * 2 * np.pi * i / T) / np.sqrt(T / 2)
        else:
            res[i] = np.cos(X * (2 * np.pi * i) / T) / np.sqrt(T / 2)
    return res.T

In [ ]:
# Computation of the test statistic
#
# X -- array of univariate observations
#
# p -- positive integer (used for basis construction)
#
def compute_test_stat_fourier(X, p, t_min=20, n_out_min=10, B=10):
    
    # Sample size
    n = X.shape[0]
    
    # Compute design matrix
    Psi = compute_design_Fourier(X, p)
    
    # Initialization
    T = np.zeros((n, n))
    
    for t in range(t_min, n):
        
        D = np.zeros(t)
        
        for tau in range(n_out_min, t-n_out_min):
            
            # Compute the best fitting parameter theta
            theta = compute_theta(Psi[:t, :], tau)
            Z = Psi[:t, :] @ theta
            
            # Use thresholding to avoid numerical issues
            Z = np.minimum(Z, B)
            Z = np.maximum(Z, -B)
            
            D[:tau] = 2 / (1 + np.exp(-Z[:tau]))
            D[tau:] = 2 / (1 + np.exp(Z[tau:]))
            D = np.log(D)
            
            # Compute statistics for each t
            # and each change point candidate tau
            T[tau, t] = tau * (t - tau) / t * (np.mean(D[:tau]) + np.mean(D[tau:]))
            
    # Array of test statistics
    S = np.max(T, axis=0)
    
    return S

### Online change point detection procedure (neural networks)

In [ ]:
class NN(nn.Module):
    def __init__(self, n_in, n_out):
        
        super(NN, self).__init__()
        self.act = nn.ReLU()
        self.fc1 = nn.Linear(n_in, 2 * n_in)
        self.fc2 = nn.Linear(2 * n_in, 3 * n_in)
        self.fc3 = nn.Linear(3 * n_in, n_out)        
    
    def forward(self, x):
        
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        
        return x

In [ ]:
# Computation of the test statistic
#
# X -- array of univariate observations
#
# p -- positive integer (used for basis construction)
#
def compute_test_stat_nn(X, t_min=20, n_out_min=10, B=10, n_epochs=200, model=NN):
    
    X = X.reshape(-1, 1)
    
    # Sample size
    n = X.shape[0]
    
    # Initialization
    T = np.zeros((n, n))
    
    for t in range(t_min, n):
    
        for tau in range(n_out_min, t-n_out_min):
            
            # Initialize neural network
            f = model(n_in=1, n_out=1)
            
            # Parameters of the optimizer
            opt = torch.optim.Adam(f.parameters(), lr=1e-1)
            
            X_t = torch.tensor(X[:t, :], dtype=torch.float32, requires_grad=True)
            
            # weights
            W = torch.cat((torch.ones(tau) * (t - tau), torch.ones(t - tau) * tau)).reshape(-1, 1)
            
            # Create "virtual" labels
            Y_t = torch.cat((torch.ones(tau), torch.zeros(t - tau))).reshape(-1, 1)
    
            # Loss function    
            loss_fn = nn.BCEWithLogitsLoss(weight=W)
            
            # Neural network training
            for epoch in range(n_epochs):
                
                loss = loss_fn(f(X_t), Y_t).mean()
                loss.backward()
                opt.step()
                opt.zero_grad()
                
            Z = f(X_t).detach().numpy().reshape(-1)
            
            # Use thresholding to avoid numerical issues
            Z = np.minimum(Z, B)
            Z = np.maximum(Z, -B)
            
            D = np.zeros(t)
            D[:tau] = 2 / (1 + np.exp(-Z[:tau]))
            D[tau:] = 2 / (1 + np.exp(Z[tau:]))
            D = np.log(D)
            
            # Compute statistics for each t
            # and each change point candidate tau
            T[tau, t] = tau * (t - tau) / t * (np.mean(D[:tau]) + np.mean(D[tau:]))
       
    # Array of test statistics
    S = np.max(T, axis=0)
    
    return S

### KLIEP online change point procedure (Sugiyama et al. (2008), Liu et al. (2013))

In [ ]:
def compute_test_stat_kliep(X, window_size=10, sigma=0.1):
    
    if len(X.shape) == 1:
        X = X.reshape(-1, 1)
    
    # More convenient notation
    b = window_size
    
    # Sample size
    n = X.shape[0]
    
    # Initialization
    T = np.zeros(n)
    
    for t in range(2*b + 1, n):
        
        # Test sample
        X_te = X[t-b:t]
        # Reference sample
        X_re = X[t-2*b:t-b]
        
        T[t] = kliep(X_te, X_re, sigma)
    
    return T

In [ ]:
def kliep(X_te, X_re, sigma):
    
    # Test sample size
    n_te = X_te.shape[0]
    # Reference sample size
    n_re = X_re.shape[0]
    
    # Compute pairwise distances
    te_te_dist = pairwise_distances(X_te)
    re_te_dist = pairwise_distances(X_re, X_te)
    
    # Compute kernel matrices
    te_te_kernel = np.exp(-0.5 * (te_te_dist / sigma)**2)
    re_te_kernel = np.exp(-0.5 * (re_te_dist / sigma)**2)
    
    # Initialize a vector of coefficients
    theta = cvx.Variable(n_te)
    
    # Objective
    obj = cvx.Maximize(cvx.sum(cvx.log(te_te_kernel @ theta)))
    
    # Constraints
    constraints = [cvx.sum(re_te_kernel @ theta) == n_re, theta >= 0]
    
    # Problem
    prob = cvx.Problem(obj, constraints)
    prob.solve()
    
    return obj.value

### An online change point detection procedure based on M-statistic (Li, Xie, Dai and Song (2015))

In [ ]:
# Compute MMD test statistic
def compute_test_stat_mmd(X, window_size=10, sigma=0.1):
    
    if len(X.shape) == 1:
        X = X.reshape(-1, 1)
    
    # More convenient notation
    b = window_size
    
    # Sample size
    n = X.shape[0]
    
    # Initialization
    T = np.zeros(n)
    
    for t in range(2*b + 1, n):
        
        # Test sample
        X_te = X[t-b:t, :]
        # Reference sample
        X_re = X[t-2*b:t-b, :]
        
        MMD_2 = mmd_squared(X_re, X_te, sigma)
        var = estimate_variance(X_re, window_size, sigma)
        
        T[t] = MMD_2 / np.sqrt(var)
    
    return T

In [ ]:
def mmd_squared(X, Y, sigma):
    
    # Sample size
    n = X.shape[0]
    
    # Compute pairwise distances
    xx_dist = pairwise_distances(X)
    xy_dist = pairwise_distances(X, Y)
    yy_dist = pairwise_distances(Y)
    
    # Compute kernel matrices
    xx_kernel = np.exp(-0.5 * (xx_dist / sigma)**2) - np.identity(n)
    xy_kernel = np.exp(-0.5 * (xy_dist / sigma)**2) - np.identity(n)
    yy_kernel = np.exp(-0.5 * (yy_dist / sigma)**2) - np.identity(n)
    
    # Compute the U-statistic
    u_stat = (np.sum(xx_kernel) - 2 * np.sum(xy_kernel) + np.sum(yy_kernel)) / n / (n - 1)
    
    return u_stat

In [ ]:
def h_squared(X, sigma):
    
    # Sample size
    n = X.shape[0]
    
    # Divide the array into four equal parts
    n_max = 4 * (n // 4)
    X_1 = X[0:n_max:4]
    X_2 = X[1:n_max:4]
    X_3 = X[2:n_max:4]
    X_4 = X[3:n_max:4]
    
    K_12 = np.exp(-0.5 * (np.linalg.norm(X_1 - X_2, axis=1) / sigma)**2)
    K_13 = np.exp(-0.5 * (np.linalg.norm(X_1 - X_3, axis=1) / sigma)**2)
    K_24 = np.exp(-0.5 * (np.linalg.norm(X_2 - X_4, axis=1) / sigma)**2)
    K_34 = np.exp(-0.5 * (np.linalg.norm(X_3 - X_4, axis=1) / sigma)**2)
    
    return np.mean((K_12 - K_13 - K_24 + K_34)**2)


# Compute the first term in the variance estimate
def h_cov(X, sigma):
    
    # Sample size
    n = X.shape[0]
    
    # Divide the array into six equal parts
    n_max = 6 * (n // 6)
    X_1 = X[0:n_max:6]
    X_2 = X[1:n_max:6]
    X_3 = X[2:n_max:6]
    X_4 = X[3:n_max:6]
    X_5 = X[4:n_max:6]
    X_6 = X[5:n_max:6]
    
    K_12 = np.exp(-0.5 * (np.linalg.norm(X_1 - X_2, axis=1) / sigma)**2)
    K_13 = np.exp(-0.5 * (np.linalg.norm(X_1 - X_3, axis=1) / sigma)**2)
    K_24 = np.exp(-0.5 * (np.linalg.norm(X_2 - X_4, axis=1) / sigma)**2)
    K_34 = np.exp(-0.5 * (np.linalg.norm(X_3 - X_4, axis=1) / sigma)**2)
    
    K_56 = np.exp(-0.5 * (np.linalg.norm(X_5 - X_6, axis=1) / sigma)**2)
    K_53 = np.exp(-0.5 * (np.linalg.norm(X_5 - X_3, axis=1) / sigma)**2)
    K_64 = np.exp(-0.5 * (np.linalg.norm(X_6 - X_4, axis=1) / sigma)**2)
    K_34 = np.exp(-0.5 * (np.linalg.norm(X_3 - X_4, axis=1) / sigma)**2)
    
    # Compute the second term in the variance estimate
    h_1234 = K_12 - K_13 - K_24 + K_34
    h_5634 = K_56 - K_53 - K_64 + K_34
    
    return np.mean((h_1234 - np.mean(h_1234)) * (h_5634 - np.mean(h_5634)))

# Variance estimate under the null hypothesis
def estimate_variance(X, window_size, sigma):
    
    if len(X.shape) == 1:
        X = X.reshape(-1, 1)
        
    # Sample size
    n = X.shape[0]
    
    # Compute the first term in the variance estimate
    h2 = h_squared(X, sigma) 
    
    # Compute the second term in the variance estimate
    h_c = h_cov(X, sigma)
    
    # Variance estimate
    var = 2 * (h2 + h_c) / window_size / (window_size - 1)
        
    return np.maximum(var, 1e-5)

### CUSUM

In [ ]:
def compute_cusum(X):
    T = np.zeros(X.shape[0])
    for n in range(1, X.shape[0]):
        t = np.zeros(n)
        for k in range(1, n):
            t[k] = abs((n - k) * X[:k].sum()   - k * X[k:n].sum() ) / np.sqrt(n * k * (n - k))
          # t = abs(np.mean(X[:k]) - np.mean(X[k:n]))
        T[n] = np.max(t)
    return T

In [ ]:
def compute_cusum_squared(X):
    T = np.zeros(X.shape[0])
    for n in range(1, X.shape[0]):
        t = np.zeros(n)
        for k in range(1, n):

            t[k] = abs((n - k) * np.power(X[:k], 2).sum()   - k * np.power(X[k:n], 2).sum() ) / np.sqrt(n * k * (n - k))
          # t = abs(np.mean(X[:k]) - np.mean(X[k:n]))
        T[n] = np.max(t)
    return T

### Threshold for p = 2, n_epochs = 50, sigma = 0.1



In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
p = 2
S_poly_list = []
S_nn_list = []
S_f_list = []

for item in range(10):
    X = sigma * randn(n)

    S_poly = compute_test_stat_linear(X, p)
    S_nn = compute_test_stat_nn(X, n_epochs=50)
    S_f = compute_test_stat_fourier(X, p)

    S_poly_list.append(S_poly)
    S_nn_list.append(S_nn)
    S_f_list.append(S_f)

In [ ]:
S_f_np = np.array(S_f_list)
df_f = pd.DataFrame(S_f_np.T, columns=range(10))
df_f.to_csv('thresholds/Fourier_p_2_sigma_0.1.csv',index=False)
threshold_p_2_f = np.round(np.max(S_f_np, axis=1).mean(), 4)

In [ ]:
S_nn_np = np.array(S_nn_list)
df_nn = pd.DataFrame(S_nn_np.T, columns=range(10))
df_nn.to_csv('thresholds/nn_n_epochs_50_sigma_0.1.csv',index=False)
threshold_p_2_nn = np.round(np.max(S_nn_np, axis=1).mean(), 4)

In [ ]:
S_poly_np = np.array(S_poly_list)
df_poly = pd.DataFrame(S_poly_np.T, columns=range(10))
df_poly.to_csv('thresholds/polynomial_p_2_sigma_0.1.csv',index=False)
threshold_p_2_poly = np.round(np.max(S_poly_np, axis=1).mean(), 4)

In [ ]:
S_f_np = pd.read_csv('thresholds/Fourier_p_2_sigma_0.1.csv').to_numpy()

In [ ]:
threshold_p_2_f = np.round(np.max(np.max(S_f_np[:,:9], axis = 0)), 4)
threshold_p_2_f

2.4917

In [ ]:
S_poly_np = pd.read_csv('thresholds/polynomial_p_2_sigma_0.1.csv').to_numpy()
threshold_p_2_poly = np.round(np.max(np.max(S_poly_np[:,:9], axis = 0)), 4)
threshold_p_2_poly

2.4594

In [ ]:
S_nn_np = pd.read_csv('thresholds/nn_n_epochs_50_sigma_0.1.csv').to_numpy()
threshold_nn = np.round(np.max(np.max(S_nn_np[:,:9], axis = 0)), 4)
threshold_nn

4.6886

In [ ]:
S_nn_np = pd.read_csv('thresholds/nn_n_epochs_50_sigma_0.1.csv').to_numpy()
threshold_nn = np.round(np.max(np.max(S_nn_np[:,:4], axis = 0)), 4)
threshold_nn

3.3494

### Threshold for p = 3, sigma = 0.1



In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
p = 3
S_poly_list = []
S_f_list = []

for item in range(10):
    X = sigma * randn(n)

    S_poly = compute_test_stat_linear(X, p)
    S_f = compute_test_stat_fourier(X, p)

    S_poly_list.append(S_poly)
    S_f_list.append(S_f)

In [ ]:
S_f_np = np.array(S_f_list)
df_f = pd.DataFrame(S_f_np.T, columns=range(10))
df_f.to_csv('thresholds/Fourier_p_3_sigma_0.1.csv',index=False)
threshold_p_3_f = np.round(np.max(S_f_np, axis=1).mean(), 4)

In [ ]:
S_poly_np = np.array(S_poly_list)
df_poly = pd.DataFrame(S_poly_np.T, columns=range(10))
df_poly.to_csv('thresholds/polynomial_p_3_sigma_0.1.csv',index=False)
threshold_p_3_poly = np.round(np.max(S_poly_np, axis=1).mean(), 4)

In [ ]:
S_poly_np = pd.read_csv('thresholds/polynomial_p_3_sigma_0.1.csv').to_numpy()
threshold_p_3_poly = np.round(np.max(np.max(S_poly_np[:,:9], axis = 0)), 4)
threshold_p_3_poly

3.9832

In [ ]:
S_f_np = pd.read_csv('thresholds/Fourier_p_3_sigma_0.1.csv').to_numpy()
threshold_p_3_f = np.round(np.max(np.max(S_f_np[:,:9], axis = 0)), 4)
threshold_p_3_f

3.9518

### Threshold for p = 6, sigma = 0.1


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
p = 6
S_poly_list = []
S_f_list = []

for item in range(10):
    X = sigma * randn(n)

    S_poly = compute_test_stat_linear(X, p)
    S_f = compute_test_stat_fourier(X, p)

    S_poly_list.append(S_poly)
    S_f_list.append(S_f)

In [ ]:
S_f_np = np.array(S_f_list)
df_f = pd.DataFrame(S_f_np.T, columns=range(10))
df_f.to_csv('thresholds/Fourier_p_6_sigma_0.1.csv',index=False)
threshold_p_6_f = np.round(np.max(S_f_np, axis=0).mean(), 4)

In [ ]:
S_poly_np = np.array(S_poly_list)
df_poly = pd.DataFrame(S_poly_np.T, columns=range(10))
df_poly.to_csv('thresholds/polynomial_p_6_sigma_0.1.csv',index=False)
threshold_p_6_poly = np.round(np.max(S_poly_np, axis=1).mean(), 4)

In [ ]:
S_poly_np = pd.read_csv('thresholds/polynomial_p_6_sigma_0.1.csv').to_numpy()
threshold_p_6_poly = np.round(np.max(np.max(S_poly_np[:,:4], axis = 0)), 4)
threshold_p_6_poly

3.9617

In [ ]:
S_f_np = pd.read_csv('thresholds/Fourier_p_6_sigma_0.1.csv').to_numpy()
threshold_p_6_f = np.round(np.max(np.max(S_f_np[:,:4], axis = 0)), 4)
threshold_p_6_f

6.4894

In [ ]:
# Number of observations
n = 150
sigma = 0.1

# Generate a Uniform on [-sigma sqrt(3), sigma sqrt(3)] sequence of observations
np.random.seed(1)
p = 6
S_poly_list = []
S_f_list = []
S_nn_list = []

for item in range(4):
    X = 2 * np.sqrt(3) * sigma * (np.random.rand(n) - 0.5)

    S_poly = compute_test_stat_linear(X, p)
    S_f = compute_test_stat_fourier(X, p)
    S_nn = compute_test_stat_nn(X, n_epochs=50)

    S_poly_list.append(S_poly)
    S_f_list.append(S_f)
    S_nn_list.append(S_nn)
    print(item)

0
1
2
3


In [ ]:
S_f_np = np.array(S_f_list)
df_f = pd.DataFrame(S_f_np.T, columns=range(4))
df_f.to_csv('thresholds/Fourier_p_6_sigma_0.1_uniform.csv',index=False)
threshold_p_6_f = np.round(np.max(np.max(S_f_np, axis=0)), 4)
threshold_p_6_f

6.8442

In [ ]:
S_poly_np = np.array(S_poly_list)
df_poly = pd.DataFrame(S_poly_np.T, columns=range(4))
df_poly.to_csv('thresholds/polynomial_p_6_sigma_0.1_uniform.csv',index=False)
threshold_p_6_poly = np.round(np.max(S_poly_np, axis=1).max(), 4)
threshold_p_6_poly

4.0411

In [ ]:
S_nn_np = np.array(S_nn_list)
df_nn = pd.DataFrame(S_nn_np.T, columns=range(4))
df_nn.to_csv('thresholds/nn_uniform.csv',index=False)
threshold_nn_uniform = np.round(np.max(S_nn_np, axis=1).max(), 4)
threshold_nn_uniform

4.3327

### Thresholds for KLIEP, CUSUM and M-statistic, sigma = 0.1

In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []
S_cusum_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.2)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.3)
    S_cusum = compute_cusum(X)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    S_cusum_list.append(S_cusum)

In [ ]:
# 1) Mean. KLIEP: 20, 0.25, MMD: 20, 0.45
# 2) Variance. KLIEP: 20, 0.15, MMD: 20, 0.15     0.33, 0.2
# 3) Distribution.

# 4) Clean.
# 5) SNR 20.
# 6) SNR 15.
# 7) SNR 10.

In [ ]:
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_20_sigma_0.1.csv',index=False)
threshold_kliep_20 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
threshold_kliep_20

6.0889

In [ ]:
S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_30_sigma_0.1.csv',index=False)
threshold_mmd_30 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
threshold_mmd_30

22.713

In [ ]:
S_cusum_np = np.array(S_cusum_list)
df_cusum = pd.DataFrame(S_cusum_np.T, columns=range(9))
df_cusum.to_csv('thresholds/cusum_sigma_0.1.csv',index=False)
threshold_cusum = np.round(np.max(np.max(S_cusum_np, axis=1)), 4)
threshold_cusum

0.4469

In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.25)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.35)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)

In [ ]:
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_25_sigma_0.1.csv',index=False)
threshold_kliep_25 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
threshold_kliep_25

5.3147

In [ ]:
S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_35_sigma_0.1.csv',index=False)
threshold_mmd_35 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
threshold_mmd_35

17.9604

In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.35)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.2)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_35_sigma_0.1.csv',index=False)
threshold_kliep_35 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_35)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_20_sigma_0.1.csv',index=False)
threshold_mmd_20 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_20)

3.8868
34.0885


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.33)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.15)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_33_sigma_0.1.csv',index=False)
threshold_kliep_33 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_33)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_15_sigma_0.1.csv',index=False)
threshold_mmd_15 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_15)

4.1893
42.9836


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.5)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.5)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_50_sigma_0.1.csv',index=False)
threshold_kliep_50 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_50)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_50_sigma_0.1.csv',index=False)
threshold_mmd_50 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_50)

2.2427
9.5924


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.4)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.45)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_40_sigma_0.1.csv',index=False)
threshold_kliep_40 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_40)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_45_sigma_0.1.csv',index=False)
threshold_mmd_45 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_45)

3.2149
11.6102


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.33)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.10)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_33_sigma_0.1.csv',index=False)
threshold_kliep_33 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_33)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_10_sigma_0.1.csv',index=False)
threshold_mmd_10 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_10)

4.1893
36.745


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(9):
    X = sigma * randn(n)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.18)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.25)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(9))
df_kliep.to_csv('thresholds/kliep_ws_20_band_18_sigma_0.1.csv',index=False)
threshold_kliep_18 = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_18)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(9))
df_mmd.to_csv('thresholds/mmd_ws_20_band_25_sigma_0.1.csv',index=False)
threshold_mmd_25 = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_25)

6.8485
28.4776


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)

S_cusum_list = []

for item in range(9):
    X = sigma * randn(n)

 
    S_cusum = compute_cusum_squared(X)
 
    S_cusum_list.append(S_cusum)

In [ ]:
S_cusum_np = np.array(S_cusum_list)
df_cusum = pd.DataFrame(S_cusum_np.T, columns=range(9))
df_cusum.to_csv('thresholds/cusum_squared_sigma_0.1.csv',index=False)
threshold_cusum_squared = np.round(np.max(np.max(S_cusum_np, axis=1)), 4)
threshold_cusum_squared

0.1465

In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(4):
    X = 2 * np.sqrt(3) * sigma * (np.random.rand(n) - 0.5)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.2)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.3)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)

In [ ]:
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(4))
df_kliep.to_csv('thresholds/kliep_ws_20_band_20_sigma_0.1_uniform.csv',index=False)
threshold_kliep_20_uniform = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_20_uniform)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(4))
df_mmd.to_csv('thresholds/mmd_ws_20_band_30_sigma_0.1_uniform.csv',index=False)
threshold_mmd_30_uniform = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_30_uniform)

4.6019
15.8071


In [ ]:
# Number of observations
n = 150

# Standard deviation of the observations
sigma = 0.1

# Generate a Gaussian sequence of observations
np.random.seed(1)
S_kliep_list = []
S_mmd_list = []

for item in range(4):
    X = 2 * np.sqrt(3) * sigma * (np.random.rand(n) - 0.5)

    S_kliep = compute_test_stat_kliep(X, window_size=20, sigma=0.45)
    S_mmd = compute_test_stat_mmd(X, window_size=20, sigma=0.6)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)

In [ ]:
S_kliep_np = np.array(S_kliep_list)
df_kliep = pd.DataFrame(S_kliep_np.T, columns=range(4))
df_kliep.to_csv('thresholds/kliep_ws_20_band_45_sigma_0.1_uniform.csv',index=False)
threshold_kliep_45_uniform = np.round(np.max(np.max(S_kliep_np, axis=1)), 4)
print(threshold_kliep_45_uniform)

S_mmd_np = np.array(S_mmd_list)
df_mmd = pd.DataFrame(S_mmd_np.T, columns=range(4))
df_mmd.to_csv('thresholds/mmd_ws_20_band_60_sigma_0.1_uniform.csv',index=False)
threshold_mmd_60_uniform = np.round(np.max(np.max(S_mmd_np, axis=1)), 4)
print(threshold_mmd_60_uniform)

1.1267
5.202


### Threshold for p = 10, n_epochs=200, sigma = sigma_1


In [ ]:
# Number of observations
n = 100

# Standard deviation of the observations
sigma = sigma_1

# Generate a Gaussian sequence of observations
np.random.seed(1)
p = 10
S_poly_list = []
S_nn_list = []
S_f_list = []

for item in range(9):
    X = sigma * randn(n)

    S_poly = compute_test_stat_linear(X, p)
    S_nn = compute_test_stat_nn(X, n_epochs=200)
    S_f = compute_test_stat_fourier(X, p)

    S_poly_list.append(S_poly)
    S_nn_list.append(S_nn)
    S_f_list.append(S_f)

In [ ]:
S_nn_np = pd.read_csv('thresholds/nn_n_epochs_200_sigma_1.csv').to_numpy()
threshold_nn_1 = np.round(np.max(np.max(S_nn_np[:,:9], axis = 0)), 4)
threshold_nn_1

In [ ]:
S_f_np = np.array(S_f_list)
df_f = pd.DataFrame(S_f_np.T, columns=range(10))
df_f.to_csv('thresholds/Fourier_p_10_sigma_1.csv',index=False)

In [ ]:
S_nn_np = np.array(S_nn_list)
df_nn = pd.DataFrame(S_nn_np.T, columns=range(10))
df_nn.to_csv('thresholds/nn_n_epochs_200_sigma_1.csv',index=False)

In [ ]:
S_poly_np = np.array(S_poly_list)
df_poly = pd.DataFrame(S_poly_np.T, columns=range(10))
df_poly.to_csv('thresholds/polynomial_p_10_sigma_1.csv',index=False)

In [ ]:
S_poly_np = pd.read_csv('thresholds/polynomial_p_10_sigma_1.csv').to_numpy()
threshold_p_10_poly_1 = np.round(np.max(np.max(S_poly_np[:,:9], axis = 0)), 4)
threshold_p_10_poly_1

0.0

In [ ]:
S_nn_np = pd.read_csv('thresholds/nn_n_epochs_200_sigma_1.csv').to_numpy()
threshold_nn_1 = np.round(np.max(np.max(S_nn_np[:,:9], axis = 0)), 4)
threshold_nn_1

0.103

In [ ]:
S_f_np = pd.read_csv('thresholds/Fourier_p_10_sigma_1.csv').to_numpy()
threshold_p_10_f_1 = np.round(np.max(np.max(S_f_np[:,:9], axis = 0)), 4)
threshold_p_10_f_1

2.4887